# Need to figure out how to confirm that the format is correct...

In [1]:
def dup_entries(df):
    duplicates = df[df.duplicated(subset=['racer_id', 'discipline'], keep=False)]
    
    if not duplicates.empty:
        print("Found duplicates for (racer_id, discipline):")
        return duplicates
    else:
        print("No duplicates found.")
        return None

# Create the connection

In [2]:
import pandas as pd
import sqlite3
from helper_functions import prep_race_results, upload_results, drop_nulls

# Set the database path to a location with write permissions
db_path = '../race_league_results.db'

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


# Upload timetrial data (just race results)
**Done**

In [6]:
import pandas as pd
import numpy as np
from helper_functions import clean_string, RACE_RESULT_COL_ORDER, upload_results

In [7]:
result_cols = ["bib", "discipline", "racer_id", "run1", "run2"]
df_load = pd.read_csv("2026/20260103FnGRaceSimResultsBoth.csv", header=None, names=result_cols)

In [8]:
df_load["racer_id"] = df_load["racer_id"].apply(clean_string)
dup_entries(df_load)
df_load.replace({'DNF': 9998,"DSQ": 9998, "DNS": 9999, pd.NA: 9999}, inplace=True)

No duplicates found.


In [6]:
# Mispelling and update mapping to current racer IDs
name_fixes = {
    "mitchperrault": "mitchperreault",
    "joannaperrault": "joannaperreault",
    "nickbalan": "nicholasbalan",
    "jeffreykansun": "jeffkansun",
    "randymilthorpe": "robertmilthorpe"
}

df_load["racer_id"] = df_load["racer_id"].replace(name_fixes)

In [10]:
df_load[df_load.racer_id == 'robertmilthorpe']

,bib,discipline,racer_id,run1,run2
7,17,SKI,robertmilthorpe,34.64,NaN


In [8]:
df_load.head(2)

,bib,discipline,racer_id,run1,run2
0,3,SKI,alexhenry,39.58,NaN
1,5,SKI,tonypetrella,34.51,NaN


In [9]:
race_date = '01/03/2026'
race_id = 260103
description = '2026 Race Sim (time trials)'

df_load['run1'] = pd.to_numeric(df_load['run1']) # errors='coerce')
df_load['run2'] = pd.to_numeric(df_load['run2']) # errors='coerce')
df_load['best_time'] = df_load[['run1', 'run2']].min(axis=1)
df_load["points"] = np.nan
df_load["team"] = np.nan
df_load["tier"] = np.nan
df_load["race_id"] = race_id

In [10]:
df_load[RACE_RESULT_COL_ORDER].head(2)

,racer_id,discipline,team,tier,run1,run2,best_time,points,race_id,bib
0,alexhenry,SKI,NaN,NaN,39.58,NaN,39.58,NaN,260103,3
1,tonypetrella,SKI,NaN,NaN,34.51,NaN,34.51,NaN,260103,5


## Review "new" racers to ensure no mispelling

HERE, one more review, THEN UPLOAD! Hmm, that didn't work for Jeff. Gotta figure out why

In [11]:
foo = pd.read_sql_query("SELECT racer_id, CONCAT(first_name, ' ', last_name) AS name from Racers;", conn)
foo

,racer_id,name
0,bradabbott,Brad Abbott
1,toriabbott,Tori Abbott
2,jayadlington,Jay Adlington
3,suealexanderash,Sue Alexander Ash
4,christineallan,Christine Allan
...,...,...
357,stevecrawford,Steve Crawford
358,torijohnston,Tori Johnston
359,tylerpridham,Tyler Pridham
360,willstewart,Will Stewart


In [15]:
bar = pd.read_sql_query("SELECT racer_id, CONCAT(first_name, ' ', last_name) AS name from Racers where lower(last_name) like '%thayer%';", conn)
bar

,racer_id,name


In [14]:
df_load[~df_load.racer_id.isin(foo.racer_id)]

,bib,discipline,racer_id,run1,run2,best_time,points,team,tier,race_id
9,20,SKI,bernieoegema,36.67,NaN,36.67,NaN,NaN,NaN,260103
36,52,SKI,kennyfreeman,97.34,NaN,97.34,NaN,NaN,NaN,260103
38,55,SKI,arilevin,32.41,NaN,32.41,NaN,NaN,NaN,260103
43,64,SKI,patricksmith,30.69,NaN,30.69,NaN,NaN,NaN,260103
45,66,SKI,courtneyscott,31.57,NaN,31.57,NaN,NaN,NaN,260103
47,74,SKI,noahschein,35.39,NaN,35.39,NaN,NaN,NaN,260103
49,76,SKI,alisonpridham,53.23,NaN,53.23,NaN,NaN,NaN,260103
50,77,SKI,cameronscott,29.68,NaN,29.68,NaN,NaN,NaN,260103
52,79,SKI,alastairconnor,27.25,NaN,27.25,NaN,NaN,NaN,260103
56,86,SKI,alexanderthalassinos,38.82,NaN,38.82,NaN,NaN,NaN,260103


## Confirmed, good to upload!

In [16]:
upload_results(
    df_load[RACE_RESULT_COL_ORDER],
    race_id,
    race_date,
    description,
    conn
)

# Update Racers table
(Use when needed)

In [1]:
import pandas as pd
import numpy as np
import sqlite3

# Set the database path to a location with write permissions
db_path = '../race_league_results.db'

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [2]:
sql = """
with base AS (SELECT a.racer_id, max(a.name) as name
FROM Teams AS a
LEFT JOIN Racers AS b
ON a.racer_id = b.racer_id
WHERE b.racer_id IS NULL
group by 1)
SELECT racer_id,
    substr(name, 1, instr(name, ' ') - 1) AS first_name,
    substr(name, instr(name, ' ') + 1) AS last_name,
    NULL as gender,
    NULL as birth_year
FROM base
"""
df = pd.read_sql(sql, conn)
df.head()

,racer_id,first_name,last_name,gender,birth_year
0,adamszakacs,Adam,Szakacs,None,None
1,andrewmoscoe,Andrew,Moscoe,None,None
2,antonioliberatore,Antonio,Liberatore,None,None
3,audunbreder,Audun,Breder,None,None
4,audunbredur,Audun,Bredur,None,None


In [3]:
df.shape

(53, 5)

In [4]:
#df.to_sql('Racers', conn, if_exists='append', index=False)
#conn.commit()

# Update team data (new year start list)!
**Done**

In [17]:
import os 

os.getcwd()

'/Users/will/Documents/github/cartar/alpine-fng.github.io/data'

In [18]:
teams_df = pd.read_csv('2026/2026_Start_List.csv')
teams_df = drop_nulls(teams_df, "name")
teams_df["racer_id"] = teams_df["name"].apply(clean_string)
teams_df.columns = ["bib", "discipline", "name", "tier", "team", "racer_id"]
teams_df["year"] = 2026

In [20]:
# Mispelling and update mapping to current racer IDs
name_fixes = {
    "mitchperrault": "mitchperreault",
    "joannaperrault": "joannaperreault",
    "nickbalan": "nicholasbalan",
    "jeffreykansun": "jeffkansun"
}

teams_df["racer_id"] = teams_df["racer_id"].replace(name_fixes)

In [22]:
teams_df[teams_df.racer_id.isin(['jeffkansun',"nicholasbalan","joannaperreault", "mitchperreault"])]

,bib,discipline,name,tier,team,racer_id,year
54,60,SKI,Mitch Perrault,7.0,Orange,mitchperreault,2026
76,85,SKI,Joanna Perrault,4.0,Orange,joannaperreault,2026
95,105,SKI,Jeffrey Kansun,2.0,Pink Powder Club,jeffkansun,2026
101,111,SKI,Nick Balan,1.0,Green Machine,nicholasbalan,2026


In [19]:
teams_df

,bib,discipline,name,tier,team,racer_id,year
0,1,SKI,Kenny Freeman,13.0,Yellow,kennyfreeman,2026
1,2,SKI,Nick Davies,13.0,Orange,nickdavies,2026
2,3,SKI,Mike Thalassinos,13.0,Red,mikethalassinos,2026
3,4,SKI,Alison Pridham,13.0,Big Eggplant Energy,alisonpridham,2026
4,5,SKI,Sachin Shah,13.0,Black,sachinshah,2026
...,...,...,...,...,...,...,...
116,140,SNBD,Linda Leistner,15.0,Black,lindaleistner,2026
117,144,SNBD,Sheri Ramshaw,15.0,Pink Powder Club,sheriramshaw,2026
118,165,SNBD,Jarid Palter,14.0,Red,jaridpalter,2026
119,188,SNBD,Jonathan Ages,NaN,NaN,jonathanages,2026


In [23]:
teams_df.to_sql('Teams', conn, if_exists='append', index=False)
conn.commit()

# Remove folks who aren't racing anymore from the teams!
(I didn't include teams for folks that aren't racing... lets see how that plays out instead of setting them to inactive)

In [7]:
sql = """
ALTER TABLE Teams
ADD COLUMN is_active BOOLEAN DEFAULT TRUE;
"""

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

In [15]:
#Ryan Wood #22 can't participate anymore. 
sql = """
UPDATE Teams
SET is_active = FALSE
where racer_id='ryanwood'
and year = 2025;
"""

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

sql = """
SELECT * FROM Teams
where racer_id = 'ryanwood'
"""
df = pd.read_sql_query(sql, conn)
df

,bib,discipline,name,tier,team,racer_id,year,is_active
0,22,SKI,Ryan Wood,9,Blue,ryanwood,2025,0


In [12]:
#Adam Grossman #43 is out for the season with an injury.  
sql = """
UPDATE Teams
SET is_active = FALSE
where racer_id='adamgrossman'
and year = 2025;
"""

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

sql = """
SELECT * FROM Teams
where racer_id='adamgrossman'
"""
df = pd.read_sql_query(sql, conn)
df

,bib,discipline,name,tier,team,racer_id,year,is_active
0,50,SKI,Adam Grossman,7,Adam Grossman,adamgrossman,2024,1
1,43,SKI,Adam Grossman,6,Purple,adamgrossman,2025,0


In [7]:
#Mctaggart is out....
sql = """
UPDATE Teams
SET is_active = FALSE
where racer_id = 'michaelmctaggart'
And year = 2025;
"""
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

sql = """
SELECT * FROM Teams
where racer_id = 'michaelmctaggart'
And year = 2025;"""
df = pd.read_sql_query(sql, conn)
df

,bib,discipline,name,tier,team,racer_id,year,is_active
0,87,SKI,Michael McTaggart,1,Black,michaelmctaggart,2025,0


# Fix team names:

In [17]:
foo = pd.read_sql_query("SELECT * FROM Teams WHERE team = 'Green' AND year = 2025 ORDER BY tier;", conn)
foo

,bib,discipline,name,tier,team,racer_id,year
0,84,SKI,Mclean Wood,1,Green,mcleanwood,2025
1,74,SKI,William Cartar,2,Green,williamcartar,2025
2,67,SKI,Brad Brock,3,Green,bradbrock,2025
3,63,SKI,Mark Laurie,4,Green,marklaurie,2025
4,57,SKI,Todd Robertson,5,Green,toddrobertson,2025
5,46,SKI,Craig Pitman,6,Green,craigpitman,2025
6,38,SKI,David Eva,7,Green,davideva,2025
7,30,SKI,Sarah Robertson,8,Green,sarahrobertson,2025
8,19,SKI,Audun Breder,9,Green,audunbreder,2025
9,12,SKI,Antonio Liberatore,10,Green,antonioliberatore,2025


In [19]:
sql = """
UPDATE Teams
SET team = 'Orange Crush'
WHERE team = 'Orange'
AND year = 2025;
"""

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

In [18]:
sql = """
UPDATE Teams
SET team = 'HULK SMASH (Green)'
WHERE team = 'Green'
AND year = 2025;
"""

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

# First race upload 2025

In [3]:
path = '2026/20260111FnGResultsBoth.csv'
race_date = '01/11/2026'
race_id = 260111
year = 2026
description = '2026 F&G Race #1'
#N_tiers=13
#N_teams=8 
max_points=10

name_fixes = {
    "mitchperrault": "mitchperreault",
    "joannaperrault": "joannaperreault",
    "nickbalan": "nicholasbalan",
    "jeffreykansun": "jeffkansun",
    "randymilthorpe": "robertmilthorpe"
}

In [4]:
results = prep_race_results(
    path,
    race_id,
    year,
    max_points,
    name_fixes,
    conn
)

/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:189: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  team_df = pd.concat([team_df, pd.DataFrame([missing_row])], ignore_index=True)
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:189: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  team_df = pd.concat([team_df, pd.DataFrame([missing_row])], ignore_index=True)
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functio

In [5]:
results.sort_values(by=["tier", "points"], ascending=[True, False], inplace=False).to_csv("test_results.csv")

In [6]:
# upload results:
#upload_results(results, race_id, race_date, description, conn)

In [ ]:
## After upload, remember to update the Index file!

# Upload second race

In [3]:
path = '2026/20260111FnGResultsBoth.csv'
race_date = '01/11/2026'
race_id = 260111
year = 2026
description = '2026 F&G Race #1'
#N_tiers=13
#N_teams=8 
max_points=10

name_fixes = {
    "mitchperrault": "mitchperreault",
    "joannaperrault": "joannaperreault",
    "nickbalan": "nicholasbalan",
    "jeffreykansun": "jeffkansun",
    "randymilthorpe": "robertmilthorpe"
}

In [4]:
results = prep_race_results(
    path,
    race_id,
    year,
    max_points,
    name_fixes,
    conn
)

/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tier_sorted.loc[tie_indices, 'points'] = tied_points
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tier_sorted.loc[tie_indices, 'points'] = tied_points
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly 

In [5]:
results.sort_values(by=["tier", "points"], ascending=[True, False], inplace=False).to_csv("test_results.csv")

In [6]:
# upload results: -> wait to upload for time confirmation from Will!!
upload_results(results, race_id, race_date, description, conn)

# Upload 3rd race

In [9]:
path = '2026/20260111FnGResultsBoth.csv'
race_date = '01/11/2026'
race_id = 260111
year = 2026
description = '2026 F&G Race #1'
#N_tiers=13
#N_teams=8 
max_points=10

name_fixes = {
    "mitchperrault": "mitchperreault",
    "joannaperrault": "joannaperreault",
    "nickbalan": "nicholasbalan",
    "jeffreykansun": "jeffkansun",
    "randymilthorpe": "robertmilthorpe"
}

In [10]:
results = prep_race_results(
    path,
    race_id,
    year,
    max_points,
    name_fixes,
    conn
)

/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tier_sorted.loc[tie_indices, 'points'] = tied_points
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tier_sorted.loc[tie_indices, 'points'] = tied_points
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly 

In [11]:
results.sort_values(by=["tier", "points"], ascending=[True, False], inplace=False).to_csv("test_results.csv")

In [6]:
# upload results: -> wait to upload for time confirmation from Will!!
upload_results(results, race_id, race_date, description, conn)

# Upload 4th race

In [3]:
path = '2026/20260111FnGResultsBoth.csv'
race_date = '01/11/2026'
race_id = 260111
year = 2026
description = '2026 F&G Race #1'
#N_tiers=13
#N_teams=8 
max_points=10

name_fixes = {
    "mitchperrault": "mitchperreault",
    "joannaperrault": "joannaperreault",
    "nickbalan": "nicholasbalan",
    "jeffreykansun": "jeffkansun",
    "randymilthorpe": "robertmilthorpe"
}

In [4]:
results = prep_race_results(
    path,
    race_id,
    year,
    max_points,
    name_fixes,
    conn
)

/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tier_sorted.loc[tie_indices, 'points'] = tied_points
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tier_sorted.loc[tie_indices, 'points'] = tied_points
/Users/will/Documents/github/cartar/alpine-fng.github.io/data/helper_functions.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '8.5' has dtype incompatible with int64, please explicitly 

In [5]:
results.sort_values(by=["tier", "points"], ascending=[True, False], inplace=False).to_csv("test_results.csv")

In [6]:
# upload results:
upload_results(results, race_id, race_date, description, conn)

## Misc work

In [23]:
order_by, race_id, year = "team, tier", 250112, 2025

sql = f"""
SELECT 
    RANK() OVER (
        ORDER BY COALESCE(race.best_time,99999) ASC
    ) AS rank,
    CASE
        WHEN team.bib is NOT NULL THEN team.bib
        ELSE race.bib
    END AS bib
    ,
    CASE
        WHEN team.name is NOT NULL THEN team.name
        WHEN names.name is NOT NULL THEN names.name
        ELSE race.racer_id
    END AS name,
    race.racer_id,
    race.discipline,
    team,
    tier,
    run1,
    run2,
    best_time,
    points
FROM (
    select racer_id, discipline, team, tier, run1, run2, best_time, points, bib
    from RaceResults
    where race_id = {race_id}
) AS race 
LEFT JOIN (
    select bib, discipline, racer_id, name
    from Teams
    where year = {year}
    AND is_active = TRUE
) AS team
ON race.racer_id = team.racer_id
AND race.discipline = team.discipline
LEFT JOIN (
    SELECT racer_id, CONCAT(first_name, ' ', last_name) AS name
    from Racers
) AS names
ON race.racer_id = names.racer_id
ORDER BY rank;
"""

df = pd.read_sql_query(sql, conn)

In [24]:
df

,rank,bib,name,racer_id,discipline,team,tier,run1,run2,best_time,points
0,1,78.0,Jeff Cox,jeffcox,SKI,Red,2.0,24.95,25.47,24.95,10.0
1,2,84.0,Mclean Wood,mcleanwood,SKI,HULK SMASH (Green),1.0,25.36,9999.00,25.36,10.0
2,3,77.0,Mike Parr,mikeparr,SKI,Purple,2.0,25.46,25.54,25.46,9.0
3,4,74.0,William Cartar,williamcartar,SKI,HULK SMASH (Green),2.0,25.74,25.49,25.49,8.0
4,5,76.0,Emma Williamson,emmawilliamson,SKI,Pink,2.0,25.50,26.05,25.50,7.0
...,...,...,...,...,...,...,...,...,...,...,...
100,98,NaN,avg_tier_points,avg_tier_points,None,Yellow,13.0,NaN,NaN,NaN,9.0
101,98,NaN,avg_tier_points,avg_tier_points,None,Pink,13.0,NaN,NaN,NaN,9.0
102,98,NaN,avg_tier_points,avg_tier_points,None,Black,1.0,NaN,NaN,NaN,7.0
103,98,NaN,avg_tier_points,avg_tier_points,None,Black,13.0,NaN,NaN,NaN,9.0


In [ ]:
#where race.best_time is not null

# Delete a race:

In [9]:
sql = """
SELECT *
FROM Races
where race_id = 250112
"""
df = pd.read_sql_query(sql, conn)
df.head()

,race_id,race_date,description
0,250112,2025-01-12 00:00:00,2025 F&G Race #1
1,250112,2025-01-12 00:00:00,2025 F&G Race #1


In [10]:
sql = """
DELETE FROM Races
where race_id = 250112;
"""

#cursor = conn.cursor()
#cursor.execute(sql)
#conn.commit()

In [11]:
sql = """
SELECT *
FROM Races
where race_id = 250112
"""
df = pd.read_sql_query(sql, conn)
df.head()

,race_id,race_date,description


In [12]:
sql = """
SELECT *
FROM RaceResults
where race_id = 250112
"""
df = pd.read_sql_query(sql, conn)
df.head()

,racer_id,discipline,team,tier,run1,run2,best_time,points,race_id,bib
0,davidmoull,SKI,HULK SMASH (Green),11.0,38.27,37.49,37.49,4.0,250112,1.0
1,davegraff,SKI,HULK SMASH (Green),11.0,35.79,34.61,34.61,9.0,250112,4.0
2,antonioliberatore,SKI,HULK SMASH (Green),10.0,37.13,36.23,36.23,4.0,250112,12.0
3,audunbreder,SKI,HULK SMASH (Green),9.0,30.39,29.97,29.97,10.0,250112,19.0
4,sarahrobertson,SKI,HULK SMASH (Green),8.0,34.37,34.56,34.37,3.0,250112,30.0


In [13]:
sql = """
DELETE FROM RaceResults
where race_id = 250112;
"""

#cursor = conn.cursor()
#cursor.execute(sql)
#conn.commit()

In [14]:
sql = """
SELECT *
FROM RaceResults
where race_id = 250112
"""
df = pd.read_sql_query(sql, conn)
df.head()

,racer_id,discipline,team,tier,run1,run2,best_time,points,race_id,bib


In [ ]:
path = 'data/2024/240218FnGResultsBoth.csv'
race_date = '02/18/2024'
race_id = 240218
description = '2024 F&G Race #4'
N_tiers=13
N_teams=8 

""" # Comment out for now
upload_new_race_results(
    path,
    race_date,
    race_id,
    description,
    N_tiers,
    N_teams,
    conn
)
"""